## Format conversion from Asciii to NetCDF

Coder: jamueller

### Make Imports

In [37]:
import pandas as pd
import numpy as np
import os
import glob
import xarray as xr

In [38]:
def status (s):
    try:
        binary = f'{int("0x%s"%s, 16):0>8b}'[::-1]
    except ValueError:
        return False, False, False, 0
        
    error = bool(int(binary[0])) or bool(int(binary[5]))
    heatingCrit = bool(int(binary[6]))
    heatingOn = bool(int(binary[7]))
    usedMemory = int(binary[1:4],2)
    return error, heatingCrit, heatingOn, usedMemory

## Get files

files in format beginning from 24.11.2023

In [39]:
ascii_folder_path = "/projekt1/ag_maahn/data_obs/hyytiala/sonic/ascii"
#netcdf_folder_path = "/projekt1/ag_maahn/data_obs_nobackup/hyytiala/sonic"
netcdf_folder_path = "/home/jamueller"

In [40]:
## here do magic with the folder

In [41]:
# use specific file for testing

testfile = ascii_folder_path+"/2025/01/01/20250101_00.txt.gz"

testfile_out = netcdf_folder_path+"/20250101_00.nc"

In [42]:
testfile

'/projekt1/ag_maahn/data_obs/hyytiala/sonic/ascii/2025/01/01/20250101_00.txt.gz'

In [43]:
expected_cols = [

    "time",'u(ms^-1)', 'v(ms^-1)', 'w(ms^-1)','TvC^-1',"STAT", "CHECK"
]



# Read the file
df = pd.read_csv(
    testfile,
    skiprows=0,         # adjust if there are header rows to skip
    header=None,        # no header row in file
    encoding='latin1',
    sep=';',
    usecols=range(7)
)

# Assign column names
df.columns = expected_cols

# Convert time manually
df['time'] = pd.to_datetime(df['time'].astype(str).str[:20], format='%Y%m%d%H%M%S%f')

df
#df_list = []

#df_combined = pd.concat(df_list)

,time,u(ms^-1),v(ms^-1),w(ms^-1),TvC^-1,STAT,CHECK
0,2025-01-01 00:00:00.030655,0.22,0.10,0.16,-3.2,CE,3C
1,2025-01-01 00:00:00.077381,0.22,0.09,0.15,-3.2,CE,37
2,2025-01-01 00:00:00.109287,0.22,0.08,0.14,-3.3,CE,36
3,2025-01-01 00:00:00.157069,0.22,0.08,0.14,-3.3,CE,36
4,2025-01-01 00:00:00.188914,0.22,0.08,0.14,-3.4,CE,31
...,...,...,...,...,...,...,...
89996,2025-01-01 00:59:59.806102,-0.42,-0.27,0.13,-4.7,CE,39
89997,2025-01-01 00:59:59.853795,-0.43,-0.25,0.14,-4.6,CE,3C
89998,2025-01-01 00:59:59.885787,-0.40,-0.23,0.15,-4.5,CE,3B
89999,2025-01-01 00:59:59.933449,-0.41,-0.18,0.17,-4.6,CE,33


In [44]:
#Add status information as new column

stat = list(map(status, df['STAT'])) #uses status function from sonic_viss_v6 notebook probabily! 

stat_df = pd.DataFrame(
    stat,
    columns=["error", "heatingCrit", "heatingOn", "usedMemory"],
    index=df.index
)

df = pd.concat([df, stat_df], axis=1)

df


,time,u(ms^-1),v(ms^-1),w(ms^-1),TvC^-1,STAT,CHECK,error,heatingCrit,heatingOn,usedMemory
0,2025-01-01 00:00:00.030655,0.22,0.10,0.16,-3.2,CE,3C,False,True,True,7
1,2025-01-01 00:00:00.077381,0.22,0.09,0.15,-3.2,CE,37,False,True,True,7
2,2025-01-01 00:00:00.109287,0.22,0.08,0.14,-3.3,CE,36,False,True,True,7
3,2025-01-01 00:00:00.157069,0.22,0.08,0.14,-3.3,CE,36,False,True,True,7
4,2025-01-01 00:00:00.188914,0.22,0.08,0.14,-3.4,CE,31,False,True,True,7
...,...,...,...,...,...,...,...,...,...,...,...
89996,2025-01-01 00:59:59.806102,-0.42,-0.27,0.13,-4.7,CE,39,False,True,True,7
89997,2025-01-01 00:59:59.853795,-0.43,-0.25,0.14,-4.6,CE,3C,False,True,True,7
89998,2025-01-01 00:59:59.885787,-0.40,-0.23,0.15,-4.5,CE,3B,False,True,True,7
89999,2025-01-01 00:59:59.933449,-0.41,-0.18,0.17,-4.6,CE,33,False,True,True,7


In [45]:
# Convert into data array and then netcdf

ds = xr.Dataset.from_dataframe(df)

ds.to_netcdf(testfile_out, engine="netcdf4")

Note: you may need to restart the kernel to use updated packages.


# next steps: 

mitteln? nein nicht mitteln! (40 m/s ) auflösung
np.datetime64

Einheiten hinzufügen 



in netcdf file speichern


tt5 statusthies status 

CE in bit umwandeln 



CE hexadezimal zahl -> in bit umwandeln  (ist bit 0 das erste oder letze) 

in quicklook 



nur eine tagesdatei pro tag -> also jahrensorder 

putty for file viewer 


quicklook rotoiene erweitern um netcdf am anfang 


hier die netcdf reinschreiben!! /projekt1/ag_maahn/data_obs_nobackup/hyytiala/sonic/2025$  




Next Steps: 

get writing permission for  projekt1/ag_maahn/data_obs_nobackup/hyytiala/sonic/2025$

make loop to create file for every day -> dann jahresordner 

include it in quicklook datei! 


check if it is a problem to work in netcdf4 (i do this because cdf3 does not allow for microseconds) 